# Great Caria: Asymmetric Causality Engine

**Goal**: Detect asymmetric causal relationships between global economies.

Core insight: Tightening propagates faster than easing. A shock from USA→China 
is not the same as China→USA. We learn these directed, asymmetric edges.

In [ ]:
# 1. Setup
!pip install torch pandas numpy scipy statsmodels tqdm matplotlib seaborn -q

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from scipy import stats
from statsmodels.tsa.stattools import grangercausalitytests
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Mount Drive & Find Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# === FIND YOUR DATA ===
print('Searching for te_*.parquet files...')
found_path = None
for root, dirs, files in os.walk('/content/drive'):
    parquets = [f for f in files if f.startswith('te_') and f.endswith('.parquet')]
    if parquets:
        print(f'Found {len(parquets)} country files in: {root}/')
        for p in parquets[:5]:
            print(f'  - {p}')
        if len(parquets) > 5:
            print(f'  ... and {len(parquets)-5} more')
        found_path = root + '/'
        break

if found_path:
    print(f'\n==> Set DATA_PATH = "{found_path}"')
else:
    print('No te_*.parquet files found!')

In [ ]:
# === SET YOUR DATA PATH HERE ===
# Copy the path from above, or set manually
DATA_PATH = '/content/drive/MyDrive/Caria/models/data/raw/'  # <-- UPDATE THIS!

# Verify
if os.path.exists(DATA_PATH):
    files = [f for f in os.listdir(DATA_PATH) if f.startswith('te_') and f.endswith('.parquet')]
    print(f'✓ Found {len(files)} country files')
else:
    print(f'✗ Path not found: {DATA_PATH}')

## 3. Load Country Data

In [ ]:
COUNTRIES = ['USA', 'CHN', 'JPN', 'DEU', 'GBR', 'FRA', 'IND', 'BRA', 'CAN', 'KOR',
             'AUS', 'MEX', 'IDN', 'ZAF', 'CHL', 'SGP', 'NLD', 'HKG', 'CHE', 'TWN', 'VNM', 'NOR']

def load_country_data(data_path, countries):
    all_data = {}
    loaded = []
    
    for country in tqdm(countries, desc="Loading"):
        filepath = f"{data_path}te_{country}.parquet"
        if not os.path.exists(filepath):
            continue
        try:
            df = pd.read_parquet(filepath)
            if 'date' in df.columns:
                df['date'] = pd.to_datetime(df['date'])
                df = df.set_index('date')
            df.columns = [f"{country}_{col}" for col in df.columns]
            all_data[country] = df
            loaded.append(country)
        except Exception as e:
            print(f"Error {country}: {e}")
    
    if not all_data:
        raise ValueError("No data loaded!")
    
    merged = pd.concat(all_data.values(), axis=1, join='outer').sort_index().ffill().dropna()
    print(f"✓ Loaded {len(loaded)} countries, {merged.shape[0]} rows, {merged.shape[1]} features")
    print(f"  Countries: {loaded}")
    return merged, loaded

df_countries, COUNTRIES = load_country_data(DATA_PATH, COUNTRIES)

## 4. Granger Causality Baseline

In [ ]:
# Find a common feature to test (e.g., 'Index', 'GDP', etc.)
sample_country = COUNTRIES[0]
features = [c.replace(f"{sample_country}_", "") for c in df_countries.columns if c.startswith(f"{sample_country}_")]
print(f"Available features: {features[:10]}...")

# Pick first available feature for Granger test
TEST_FEATURE = features[0] if features else 'value'
print(f"Using feature: {TEST_FEATURE}")

In [ ]:
def compute_granger_matrix(df, countries, feature, max_lag=4):
    n = len(countries)
    gc = np.ones((n, n))
    
    for i, c1 in enumerate(tqdm(countries, desc="Granger")):
        for j, c2 in enumerate(countries):
            if i == j: continue
            col1, col2 = f"{c1}_{feature}", f"{c2}_{feature}"
            if col1 not in df.columns or col2 not in df.columns: continue
            try:
                data = df[[col2, col1]].dropna()
                if len(data) < 100: continue
                result = grangercausalitytests(data, maxlag=max_lag, verbose=False)
                gc[i, j] = min([result[l][0]['ssr_ftest'][1] for l in range(1, max_lag+1)])
            except: pass
    return gc

gc_matrix = compute_granger_matrix(df_countries, COUNTRIES, TEST_FEATURE)
gc_sig = 1 - gc_matrix
np.fill_diagonal(gc_sig, 0)

plt.figure(figsize=(12, 10))
sns.heatmap(gc_sig, xticklabels=COUNTRIES, yticklabels=COUNTRIES, cmap='Reds', annot=True, fmt='.2f')
plt.title('Granger Causality: i → j')
plt.tight_layout()
plt.show()

## 5. Asymmetric Causal Transformer

In [ ]:
class AsymmetricCausalAttention(nn.Module):
    def __init__(self, d_model, n_heads=4, dropout=0.1):
        super().__init__()
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        self.W_cause = nn.Linear(d_model, d_model)
        self.W_effect = nn.Linear(d_model, d_model)
        self.W_value = nn.Linear(d_model, d_model)
        self.W_out = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.scale = np.sqrt(self.d_k)
    
    def forward(self, x, return_attention=False):
        B, N, D = x.shape
        Q = self.W_cause(x).view(B, N, self.n_heads, self.d_k).transpose(1, 2)
        K = self.W_effect(x).view(B, N, self.n_heads, self.d_k).transpose(1, 2)
        V = self.W_value(x).view(B, N, self.n_heads, self.d_k).transpose(1, 2)
        
        attn = F.softmax(torch.matmul(Q, K.transpose(-2, -1)) / self.scale, dim=-1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, V).transpose(1, 2).contiguous().view(B, N, D)
        out = self.W_out(out)
        
        if return_attention:
            return out, attn.mean(dim=1)
        return out

class TemporalEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, 2, batch_first=True, bidirectional=True)
        self.proj = nn.Linear(hidden_dim * 2, hidden_dim)
    
    def forward(self, x):
        B, T, N, F = x.shape
        x = x.permute(0, 2, 1, 3).reshape(B * N, T, F)
        out, _ = self.lstm(x)
        out = self.proj(out[:, -1, :])
        return out.view(B, N, -1)

class AsymmetricCausalFormer(nn.Module):
    def __init__(self, n_countries, n_features, d_model=64, n_heads=4, n_layers=3):
        super().__init__()
        self.temporal = TemporalEncoder(n_features, d_model)
        self.attns = nn.ModuleList([AsymmetricCausalAttention(d_model, n_heads) for _ in range(n_layers)])
        self.norms = nn.ModuleList([nn.LayerNorm(d_model) for _ in range(n_layers)])
        self.pred = nn.Sequential(nn.Linear(d_model, d_model), nn.ReLU(), nn.Dropout(0.1), nn.Linear(d_model, 1))
    
    def forward(self, x, return_attention=False):
        h = self.temporal(x)
        attns = []
        for attn, norm in zip(self.attns, self.norms):
            if return_attention:
                h_new, a = attn(h, True)
                attns.append(a)
            else:
                h_new = attn(h)
            h = norm(h + h_new)
        preds = self.pred(h).squeeze(-1)
        if return_attention:
            return preds, torch.stack(attns).mean(dim=0)
        return preds

print("Model defined ✓")

## 6. Create Dataset

In [ ]:
class CountryDataset(Dataset):
    def __init__(self, df, countries, seq_len=60):
        self.seq_len = seq_len
        self.countries = countries
        
        # Find common features
        all_suffixes = None
        for c in countries:
            suffixes = set(col.replace(f"{c}_", "") for col in df.columns if col.startswith(f"{c}_"))
            all_suffixes = suffixes if all_suffixes is None else all_suffixes & suffixes
        self.features = sorted(list(all_suffixes))
        self.n_features = len(self.features)
        print(f"Using {self.n_features} common features")
        
        # Build tensor
        T = len(df)
        tensor = np.zeros((T, len(countries), self.n_features))
        for i, c in enumerate(countries):
            for j, f in enumerate(self.features):
                col = f"{c}_{f}"
                if col in df.columns:
                    tensor[:, i, j] = df[col].values
        
        # Normalize
        self.mean = tensor.mean(axis=0, keepdims=True)
        self.std = tensor.std(axis=0, keepdims=True) + 1e-8
        tensor = (tensor - self.mean) / self.std
        
        # Targets: next-step change in first feature
        targets = np.diff(tensor[:, :, 0], axis=0)
        
        # Create sequences
        self.X, self.y = [], []
        for t in range(seq_len, T - 1):
            self.X.append(tensor[t-seq_len:t])
            self.y.append(targets[t])
        self.X, self.y = np.array(self.X), np.array(self.y)
        print(f"Dataset: {len(self.X)} samples")
    
    def __len__(self): return len(self.X)
    def __getitem__(self, i): return torch.FloatTensor(self.X[i]), torch.FloatTensor(self.y[i])

dataset = CountryDataset(df_countries, COUNTRIES, seq_len=60)
train_size = int(0.8 * len(dataset))
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)
print(f"Train: {len(train_data)}, Test: {len(test_data)}")

## 7. Train Model

In [ ]:
model = AsymmetricCausalFormer(len(COUNTRIES), dataset.n_features, d_model=64, n_heads=4, n_layers=3).to(device)
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 50)
criterion = nn.MSELoss()

for epoch in range(50):
    model.train()
    train_loss = sum(criterion(model(X.to(device)), y.to(device)).backward() or criterion(model(X.to(device)), y.to(device)).item() 
                     for X, y in train_loader) / len(train_loader) if False else 0
    
    # Proper training loop
    train_loss = 0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        loss = criterion(model(X), y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = sum(criterion(model(X.to(device)), y.to(device)).item() for X, y in test_loader) / len(test_loader)
    scheduler.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: Train={train_loss:.6f}, Test={test_loss:.6f}")

torch.save(model.state_dict(), 'great_caria_asymmetric.pth')
print("Model saved ✓")

## 8. Extract & Visualize Asymmetric Causality

In [ ]:
model.eval()
all_attn = []
with torch.no_grad():
    for X, _ in test_loader:
        _, attn = model(X.to(device), return_attention=True)
        all_attn.append(attn.cpu().numpy())

causality = np.concatenate(all_attn, axis=0).mean(axis=0)
asymmetry = np.abs(causality - causality.T)

print(f"Asymmetry Score: {asymmetry.mean():.4f} (0=symmetric, higher=more asymmetric)")

fig, axes = plt.subplots(1, 2, figsize=(18, 7))
sns.heatmap(causality, xticklabels=COUNTRIES, yticklabels=COUNTRIES, cmap='Blues', ax=axes[0], annot=True, fmt='.2f')
axes[0].set_title('Learned Causality: i → j')
sns.heatmap(asymmetry, xticklabels=COUNTRIES, yticklabels=COUNTRIES, cmap='Reds', ax=axes[1], annot=True, fmt='.2f')
axes[1].set_title('Asymmetry: |A[i,j] - A[j,i]|')
plt.tight_layout()
plt.savefig('great_caria_causality.png', dpi=150)
plt.show()

## 9. Top Asymmetric Pairs

In [ ]:
pairs = []
for i in range(len(COUNTRIES)):
    for j in range(i+1, len(COUNTRIES)):
        i_to_j, j_to_i = causality[i,j], causality[j,i]
        dominant = f"{COUNTRIES[i]} → {COUNTRIES[j]}" if i_to_j > j_to_i else f"{COUNTRIES[j]} → {COUNTRIES[i]}"
        ratio = max(i_to_j, j_to_i) / (min(i_to_j, j_to_i) + 1e-6)
        pairs.append({'pair': dominant, 'asymmetry': asymmetry[i,j], 'ratio': ratio})

pairs = sorted(pairs, key=lambda x: x['asymmetry'], reverse=True)

print("\n" + "="*50)
print("TOP 10 ASYMMETRIC CAUSAL RELATIONSHIPS")
print("="*50)
for i, p in enumerate(pairs[:10]):
    print(f"{i+1}. {p['pair']} (asymmetry: {p['asymmetry']:.4f}, ratio: {p['ratio']:.1f}x)")

## 10. Export for Frontend

In [ ]:
import json

output = {
    'modelVersion': 'Great Caria v1.0 - Asymmetric Causality',
    'countries': COUNTRIES,
    'causality_matrix': causality.tolist(),
    'asymmetry_matrix': asymmetry.tolist(),
    'top_pairs': pairs[:10],
    'stats': {'avg_asymmetry': float(asymmetry.mean()), 'max_asymmetry': float(asymmetry.max())}
}

with open('great_caria_signals.json', 'w') as f:
    json.dump(output, f, indent=2)

print("Exported to great_caria_signals.json ✓")